# Fierce Research ATR Position Sizing For Stocks


Sources: 
- For ATR: https://www.investopedia.com/terms/a/atr.asp
- For Position sizing using 2x ATR: https://www.thebalance.com/how-average-true-range-atr-can-improve-trading-4154923

#### Import Packages

In [1]:
import numpy as np
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

#### Import Data

In [2]:
import eikon as tr
tr.set_app_key('DEFAULT_CODE_BOOK_APP_KEY')

stock = 'FVL.TO'

df1, e= tr.get_data([stock],['tr.priceclose.date','tr.priceclose','tr.open','tr.high','tr.low'],
                       {'SDate':-50,'EDate':0,'Frq':'D'})

df1['Date'] = pd.to_datetime(df1['Date'])
df1.set_index('Date', inplace=True) 
df1.index = df1.index.map(lambda t: t.strftime('%Y-%m-%d'))
df1.columns = map(str.lower, df1.columns)
df1 = df1.drop_duplicates()
df1 = df1.round(2)
df1.tail()

,instrument,price close,price open,price high,price low
2020-07-03,FVL.TO,1.29,1.30,1.34,1.27
2020-07-06,FVL.TO,1.32,1.30,1.38,1.29
2020-07-07,FVL.TO,1.29,1.33,1.38,1.29
2020-07-08,FVL.TO,1.32,1.32,1.38,1.30
2020-07-09,FVL.TO,1.20,1.30,1.33,1.11


#### True Range

The TR for a given trading period is the greatest of the following:

- Current high minus the previous close
- Current low minus the previous close
- Current high minus the current low

In [3]:
df1['price close shifted'] = df1['price close'].shift(1)

In [4]:
df1['high minus previous low'] = df1['price high']-df1['price close shifted']

In [5]:
df1['low minus previous close'] = df1['price low']-df1['price close shifted']

In [6]:
df1['curent high minus current low'] = df1['price high']-df1['price low']

In [7]:
df1['true range'] = abs(df1[['high minus previous low','low minus previous close','curent high minus current low']]).max(axis=1)

#### Average True Range

In [14]:
df1['atr'] = df1['true range'].rolling(14).mean()
df1.tail(1).round(2)

,instrument,price close,price open,price high,price low,price close shifted,high minus previous low,low minus previous close,curent high minus current low,true range,atr,2x atr,stop loss,$ at risk,recommended position size
2020-07-09,FVL.TO,1.2,1.3,1.33,1.11,1.32,0.01,-0.21,0.22,0.22,0.15,0.3,0.9,250.0,829.38


#### 2x ATR For Stop Loss Level

In [9]:
df1['2x atr'] = df1['atr']*2

In [10]:
df1['stop loss'] = df1['price close']-df1['2x atr']
df1.iloc[-1:,10:].round(2)

,atr,2x atr,stop loss
2020-07-09,0.15,0.3,0.9


#### % Of Capital At Risk

In [11]:
df1['$ at risk'] = (.005*50000)

In [12]:
df1['recommended position size'] = df1['$ at risk']/df1['2x atr']

#### Go To Market Position Size

In [13]:
df1['recommended position size'][-1].round()

829.0

Good luck with the market! Guillaume, Fierce Research